# 🧠 Using Extreme Value Theory (EVT) to Estimate the Worst-Case Loss in Bitcoin (BTC)
**Author: Phiphat Chomchit**

---

## 🔍 Why EVT?

Bitcoin returns 

$$
r_t = \ln\left(\frac{P_t}{P_{t-1}}\right)
$$

exhibit:

- High kurtosis (fat tails),
- Volatility clustering,
- Non-normal behavior.

Standard models like Gaussian VaR **underestimate losses**. EVT focuses on tail events, making it ideal for modeling **extreme crashes**.

---

## 📐 EVT Approaches: Theory & Formulas

### 1. Block Maxima (BM) — GEV Distribution

Divide data into $n$ blocks (e.g., monthly), extract maximum return from each:

$$
M_n = \max\{X_1, X_2, \dots, X_n\}
$$

The **Generalized Extreme Value (GEV)** distribution:

$$
G(z) = \exp \left\{ -\left[1 + \xi\left(\frac{z - \mu}{\sigma}\right)\right]^{-1/\xi} \right\}, \quad \text{for } 1 + \xi\left(\frac{z - \mu}{\sigma}\right) > 0
$$

- $\mu$: location  
- $\sigma > 0$: scale  
- $\xi$: shape (tail index)

---

### 2. Peak Over Threshold (POT) — Generalized Pareto Distribution (GPD)

Focus on **tail exceedances**: values above a high threshold $u$.  
Define exceedances:

$$
Y = X - u \quad \text{such that } X > u
$$

Then model $Y$ using the **GPD**:

$$
F(y) = 1 - \left(1 + \frac{\xi y}{\beta} \right)^{-1/\xi}, \quad y > 0, \quad \text{where } 1 + \frac{\xi y}{\beta} > 0
$$

- $\xi$: shape — heavy tails when $\xi > 0$  
- $\beta > 0$: scale

---

## ✅ Risk Measures with EVT

Let:

- $N$: total number of observations  
- $N_u$: number of exceedances over $u$  
- $F_u = \frac{N_u}{N}$: empirical exceedance probability  
- $\alpha$: confidence level (e.g., 0.99)

### Value at Risk (VaR):

$$
\text{VaR}_\alpha = u + \frac{\beta}{\xi} \left[ \left( \frac{1 - \alpha}{F_u} \right)^{-\xi} - 1 \right]
$$

### Expected Shortfall (ES):

$$
\text{ES}_\alpha = \frac{\text{VaR}_\alpha}{1 - \xi} + \frac{\beta - \xi u}{1 - \xi}
$$

---

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import genpareto

In [2]:
# Download BTC daily prices
btc = yf.download('BTC-USD', start='2018-01-01', end='2024-12-31')
btc['Return'] = np.log(btc['Close'] / btc['Close'].shift(1))  # r_t = log(P_t / P_{t-1})
returns = btc['Return'].dropna()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [3]:
# Choose 5% left-tail threshold u
threshold = returns.quantile(0.05)
exceedances = threshold - returns[returns < threshold]  # Y = u - X, flipped to model right tail of losses

In [4]:
threshold

np.float64(-0.0558938532493803)

In [5]:
# Fit Generalized Pareto Distribution to exceedances
shape, loc, scale = genpareto.fit(exceedances)
n = len(returns)
nu = len(exceedances)
alpha = 0.97
Fu = nu / n  # empirical exceedance probability

# EVT formula: Value-at-Risk
VaR = threshold - (scale / shape) * (((1 - alpha) / Fu) ** (-shape) - 1)

# EVT formula: Expected Shortfall
ES = (VaR + (scale - shape * (threshold - VaR)) / (1 - shape))

print(f"Shape parameter ξ = {shape:.4f}")
print(f"Scale parameter β = {scale:.4f}")
print(f"Threshold u = {threshold:.4%}")
print(f"{alpha*100}% Value at Risk (VaR): {VaR:.2%}")
print(f"{alpha*100}% Expected Shortfall (ES): {ES:.2%}")

Shape parameter ξ = 1.4908
Scale parameter β = 0.0128
Threshold u = -5.5894%
97.0% Value at Risk (VaR): -6.57%
97.0% Expected Shortfall (ES): -6.19%
